In [ ]:
import os
import glob
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import astroalign as aa
import warnings
import sep
from astropy.table import Table
from astropy.wcs import WCS
import astropy.stats as stats
import wcs_solve
from astropy import time, coordinates as coord, units as u
from PyAstronomy import pyasl
from astropy.utils import iers
import photutils as pu
import datetime
from astropy.coordinates import match_coordinates_sky
from astropy.coordinates import SkyCoord
iers.conf.auto_download = False
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from astropy.visualization import simple_norm
from astropy.table import QTable
from astropy.table import Table
from astropy.stats import sigma_clipped_stats
from astropy.nddata import NDData
from photutils.psf import EPSFBuilder
from photutils.psf import extract_stars
from photutils.detection import IRAFStarFinder, DAOStarFinder
from photutils.psf import PSFPhotometry
from photutils.psf import IntegratedGaussianPRF
from scipy.stats import linregress
from pycse import regress
from photutils import CircularAperture, aperture_photometry
import re

In [ ]:
# 自定义排序规则
def custom_sort(file_name):
    # 提取文件名中的数字部分
    file_number = int(''.join(filter(str.isdigit, file_name)))

    # 处理文件名中末尾数字为1后面的数字为10的情况
    if file_number == 1:
        # 如果末尾数字为1，检查下一个字符是否是0
        next_index = file_name.index(str(file_number)) + 1
        if next_index < len(file_name) and file_name[next_index] == '0':
            # 如果下一个字符是0，则将文件末尾数字调整为10
            file_number = 10

    return file_number

In [ ]:
#定义一个显示图像的函数
def Showimg(filename):

    data=fits.getdata(filename)
    #对比度
    vmin=np.percentile(data,1)
    vmax=np.percentile(data,99)
    plt.figure(figsize=(8,6))
    plt.imshow(data,cmap='gray',vmin=vmin,vmax=vmax)
    plt.title(filename)
    plt.axis('off')

In [ ]:
#输入参数为本底文件名构成的列表，本底会保存在reddir下
def BiasCombine(biasfnlst):

    print(f'正在合并本底...\n本底图像共{len(biasfnlst)}张')

    ny,nx=fits.getval(biasfnlst[0],'NAXIS2'),fits.getval(biasfnlst[0],'NAXIS1')
    print(f'图像尺寸={nx}*{ny}')

    bias_cube=np.empty((len(biasfnlst),ny,nx),dtype='float32')
    hdr=fits.getheader(biasfnlst[0])

    for i in range(len(biasfnlst)):
        bias_cube[i]=fits.getdata(biasfnlst[i])
        hdr[f'bias{i+1}']=biasfnlst[i]

    bias=np.float32(np.nanmedian(bias_cube,axis=0))

    biaspath=f'{reddir}/'

    fits.writeto(f'{biaspath}bias.fits',data=bias,header=hdr,overwrite=True)
    print(f'本底已写入{biaspath}bias.fit')

In [ ]:
#输入参数为平场文件名构成的列表，保存的平场文件名
def FlatCombine(flatfnlst,filename):

    print(f'正在合并平场...\n平场图像共{len(flatfnlst)}张')

    ny,nx=fits.getval(flatfnlst[0],'NAXIS2'),fits.getval(flatfnlst[0],'NAXIS1')
    print(f'图像尺寸={nx}*{ny}')

    flat_cube=np.empty((len(flatfnlst),ny,nx),dtype='float32')
    hdr=fits.getheader(flatfnlst[0])

    bias=fits.getdata(f'{reddir}/bias.fits')

    for i in range(len(flatfnlst)):
        flat=fits.getdata(flatfnlst[i])
        flat=1.*flat-bias
        flat_median=np.nanmedian(flat)
        flat_cube[i]=flat/flat_median
        hdr[f'flat{i+1}']=flatfnlst[i]

    flatpath=f'{reddir}/{filename}'
    flat=np.float32(np.nanmedian(flat_cube,axis=0))
    fits.writeto(flatpath,data=flat,header=hdr,overwrite=True)

    print(f'平场已写入{flatpath}')

In [ ]:
#输入参数为科学图像文件名构成的列表，平场文件名
def CCDproc(lightfnlst,flatname):

    print(f'正在改正图像...\n科学图像共{len(lightfnlst)}张')

    ny,nx=fits.getval(lightfnlst[0],'NAXIS2'),fits.getval(lightfnlst[0],'NAXIS1')
    print(f'图像尺寸={nx}*{ny}')

    bias=fits.getdata(f'{reddir}/bias.fits')
    flat=fits.getdata(f'{reddir}/{flatname}')

    for i in range(len(lightfnlst)):
        data=fits.getdata(lightfnlst[i])
        hdr=fits.getheader(lightfnlst[i])
        data=(1.*data-bias)/flat

        newname=f"{reddir}/{lightfnlst[i].split('/')[-1].split('.fit')[0]}_bf.fits"

        fits.writeto(newname,data=data,header=hdr,overwrite=True)

        print(f'图像保存至{newname}\n完成 {i+1}/{len(lightfnlst)}')

In [ ]:
#输入参数为需要测光的图像文件名构成的列表，半高全宽，探测阈值，是否有wcs=True/False
def Phot(imgfnlst,fwhm,threshold,wcs=False):

    for i in range(len(imgfnlst)):
        
        print(f'开始测光{imgfnlst[i]}\n{i+1}/{len(imgfnlst)}')
        data=np.array(fits.getdata(imgfnlst[i]),dtype=np.float32)
        hdr=fits.getheader(imgfnlst[i])
        bkg=sep.Background(data)
        data_sub=data-bkg
        objects=sep.extract(data_sub,threshold,err=bkg.globalrms)
        print(f'找到{len(objects)}颗星')
        flux,fluxerr,flag=sep.sum_circle(data_sub,objects['x'],objects['y'],
                                        3.*fwhm,err=bkg.globalrms,gain=1.0)
        mag=-2.5*np.log10(flux)
        snr=flux/fluxerr
        magerr=2.5*np.log10((flux+fluxerr)/flux)

        if wcs==False:
            
            cat=Table()

            cat['x']=objects['x']
            cat['y']=objects['y']
            cat['flux']=flux
            cat['fluxerr']=fluxerr
            cat['mag']=mag
            cat['magerr']=magerr
            cat['snr']=snr

            cat=cat[np.argsort(cat['mag'])]

            cat_hl=fits.HDUList([
                fits.PrimaryHDU(header=hdr),
                fits.BinTableHDU(data=cat)
            ])
            
            newname=f'{os.path.splitext(imgfnlst[i])[0]}_cat.fits'
            cat_hl.writeto(newname,overwrite=True)
        
        if wcs==True:
            w=WCS(hdr)

            x=objects['x']
            y=objects['y']
            sky=w.pixel_to_world(x,y)

            ra=np.round(sky.ra.deg,3)
            dec=np.round(sky.dec.deg,3)

            cat=Table()

            cat['x']=x
            cat['y']=y
            cat['Ra']=ra
            cat['Dec']=dec
            cat['flux']=flux
            cat['fluxerr']=fluxerr
            cat['mag']=mag
            cat['magerr']=magerr
            cat['snr']=snr

            cat=cat[np.argsort(cat['mag'])]

            cat_hl=fits.HDUList([
                fits.PrimaryHDU(header=hdr),
                fits.BinTableHDU(data=cat)
            ])

            newname=f'{os.path.splitext(imgfnlst[i])[0]}_cat.fits'
            cat_hl.writeto(newname,overwrite=True)

In [ ]:
def Solve(catfn):
    # 使用os.path.splitext拆分文件名和扩展名
    file_name, file_extension = os.path.splitext(catfn)
    # 获取文件名的主体部分（去除扩展名）
    file_main = os.path.basename(file_name)
    
    catfnwcs = f'{file_name}_wcs.fits'  # 组合完整的 WCS 文件名

    # 运行 Astrometry.net 客户端
    os.system(f'python3 client.py -k bgqyiflmwpyxfscc -u "{catfn}" --newfits="{catfnwcs}"')

    # 读取原始表格数据
    rawtable = Table.read(catfn)
    # 读取 WCS 文件的头部信息
    hdr = fits.getheader(catfnwcs)

    # 将像素坐标转换为天球坐标
    w = WCS(hdr)
    x = rawtable['x']
    y = rawtable['y']
    sky = w.pixel_to_world(x, y)
    ra = sky.ra.deg
    dec = sky.dec.deg

    # 添加 RA 和 Dec 列到表格中
    rawtable['ra'] = ra
    rawtable['dec'] = dec

    # 创建带有新表格的 HDU 列表
    cat_hl = fits.HDUList([
        fits.PrimaryHDU(header=hdr),
        fits.BinTableHDU(data=rawtable)
    ])

    # 将新的 HDU 列表写入 WCS 文件
    cat_hl.writeto(catfnwcs, overwrite=True)

In [ ]:
def FitsHeaderCor(imglst):
    # 读取改正后图像header中的HJD
    mjd = np.empty(len(imglst), np.float64)
    hjd = np.empty(len(imglst), np.float64)
    jd = np.empty(len(imglst), np.float64)
    bjd = np.empty(len(imglst), np.float64)

    for k in range(len(imglst)):
        data_hd_list = fits.open(imglst[k])
        data = data_hd_list[0].data
        header = data_hd_list[0].header
        # 观测站，观测目标
        obs_site = coord.EarthLocation(lon=obs_location[0], lat=obs_location[1],
                                height = obs_location[2])
        obj_coord = coord.SkyCoord(obj_radec[0], obj_radec[1], unit=(u.hour, u.deg),
                            frame='icrs')
        # 时间计算         
        obs_datetime = header['DATE-OBS']
        obs_jd = time.Time(obs_datetime, format='isot', scale='utc', location=obs_site)
        # 地球到目标的光程差
        light_travel_time_bary = obs_jd.light_travel_time(obj_coord)
        jd[k] = obs_jd.jd
        mjd[k] = obs_jd.mjd
        bjd[k] = (obs_jd.tdb + light_travel_time_bary).jd
        hjd[k] = pyasl.helio_jd(jd[k]-2.4e6, obj_coord.ra, obj_coord.dec) + 2.4e6
        header["MJD"] = mjd[k]
        header["BJD"] = bjd[k]
        header["HJD"] = hjd[k]
        # 保存新的fits
        corrected_header_list = fits.HDUList([fits.PrimaryHDU(data=data, 
                                                            header=header)])
        corrected_header_list.writeto(imglst[k],overwrite=True)
        # 关闭fits
        data_hd_list.close()

In [ ]:
def WCS_header_Cor(imgflst,wcs_file):
    data = fits.getdata(imgflst)
    header = fits.getheader(imgflst)
    wcs_header=fits.getheader(wcs_file)
    fits.writeto(imgflst,header=header,data=data,overwrite=True)
    with fits.open(imgflst,mode='update') as hdu:
        hdu[0].header.update(wcs_header)
        hdu.flush()

In [ ]:
#以TIC288735205.01的数据为例，设置初始文件目录与输出文件目录
obs_location = [117.566 * u.deg, 40.383 * u.deg, 957 * u.m] # 观测点经度、纬度、高度
warnings.filterwarnings('ignore')
date='20240617zj' # 观测日期
obj_j='V554Dra' # 目标原名称
target = 'V554Dra'
obj_radec = ['16:41:05.9', '+60:36:22.3'] # 目标源坐标
ra_deg = 250.274
dec_deg = 60.606

rawdir=f'/data/transient/mym/125ed_data/{date}'
reddir=f'/data/transient/mym/125ed_red/{date}'

os.makedirs(reddir, exist_ok=True)

#读取rawdir中的文件夹，定义本底、平场、科学图像的列表变量
biasfnlst=glob.glob(f'{rawdir}/*bias*')

# flatfnlst_filter1=glob.glob(f'{rawdir}/FLAT/*_R_*')
flatfnlst_filter2=glob.glob(f'{rawdir}/*flat*V*')
# flatfnlst_filter3=glob.glob(f'{rawdir}/FLAT/*_B_*')

# lightfnlst_filter1=glob.glob(f'{rawdir}/LIGHT/{target}*_R_*')
lightfnlst_filter2=glob.glob(f'{rawdir}/*{target}*V*')
# lightfnlst_filter3=glob.glob(f'{rawdir}/LIGHT/{target}*_B_*')
# 检查列表
print(biasfnlst)
# print(flatfnlst_filter1)
print(flatfnlst_filter2)
# print(flatfnlst_filter3)
# print(lightfnlst_filter1)
print(lightfnlst_filter2)
# print(lightfnlst_filter3)

In [ ]:
# 合并本底
BiasCombine(biasfnlst)

# 检查本底图像
Showimg(f'{reddir}/bias.fits')

In [ ]:
# 合并平场（不同滤光片的平场要分别合并）
# FlatCombine(flatfnlst_filter1,'flat_R.fits')
FlatCombine(flatfnlst_filter2,'flat_G.fits')
# FlatCombine(flatfnlst_filter3,'flat_B.fits')
# FlatCombine(flatfnlst_filter4,'flat_R+.fits')

# 检查平场图像
# Showimg(f'{reddir}/flat_R.fits')
Showimg(f'{reddir}/flat_G.fits')
# Showimg(f'{reddir}/flat_B.fits')
# Showimg(f'{reddir}/flat_R+.fits')

In [ ]:
# 减本底、除平场
# CCDproc(lightfnlst_filter1,'flat_R.fits')
CCDproc(lightfnlst_filter2,'flat_G.fits')
# CCDproc(lightfnlst_filter3,'flat_B.fits')
# CCDproc(lightfnlst_filter4,'flat_R+.fits')

In [ ]:
#获得图像文件名列表
# imgfnlst_filter1=glob.glob(f'{reddir}/{target}*_R_*bf.fits')
imgfnlst_filter2=glob.glob(f'{reddir}/*{target}*V_bf.fits')
# imgfnlst_filter3=glob.glob(f'{reddir}/{target}*_B_*bf.fits')
# imgfnlst_filter4=glob.glob(f'{reddir}/{target}*_R+_*bf.fits')

# print(imgfnlst_filter1)
print(imgfnlst_filter2)
# print(imgfnlst_filter3)
# print(imgfnlst_filter4)

In [ ]:
# FitsHeaderCor(comfnlst_filter1)
FitsHeaderCor(imgfnlst_filter2)
# FitsHeaderCor(comfnlst_filter3)
# FitsHeaderCor(comfnlst_filter4)

In [ ]:
# # Phot(imgfnlst_filter1,fwhm=2,threshold=2)
Phot(imgfnlst_filter2,fwhm=3,threshold=3)
# # Phot(imgfnlst_filter3,fwhm=2,threshold=2)
# # Phot(imgfnlst_filter4,fwhm=2,threshold=2)

In [ ]:
catfnlst_filter2 = sorted(glob.glob(f'{reddir}/*{target}*V*bf_cat.fits'))
print(catfnlst_filter2)

In [ ]:
for files in  catfnlst_filter2:
    Solve(files)

In [ ]:
catfnlst_filter2 = glob.glob(f'{reddir}/*{target}*V*bf_cat_wcs.fits')
print(catfnlst_filter2)

In [ ]:
Table.read(catfnlst_filter2[0])

In [ ]:
#显示找星结果
data = fits.getdata(imgfnlst_filter2[99])
header = fits.getheader(catfnlst_filter2[99])
wcs = WCS(header)

# 读取星表数据
table = Table.read(catfnlst_filter2[99])
x = table['x']
y = table['y']

# 计算数据的统计量以用于图像显示
mean = np.mean(data)
std = np.std(data)
vmin = np.percentile(data, 5)
vmax = np.percentile(data, 95)

# 指定的 RA 和 Dec
radeg = ra_deg  # 替换为实际的 RA 值
decdeg = dec_deg  # 替换为实际的 Dec 值

# 将 RA 和 Dec 转换为像素坐标
sky_coord = SkyCoord(radeg * u.deg, decdeg * u.deg, frame='icrs')
pix_coord = sky_coord.to_pixel(wcs)

# 绘制图像
plt.figure(figsize=(8, 6))
plt.imshow(data, cmap='gray', vmin=vmin, vmax=vmax)
plt.scatter(x, y, color='none', edgecolors='red', label='Detected Stars')
plt.scatter(pix_coord[0], pix_coord[1], color='none', edgecolors='green', s=100, label='Target Star')

# 去掉坐标轴
plt.axis('off')
plt.legend()
plt.show()

In [ ]:
import numpy as np
from collections import Counter
from astropy.io import fits
from astropy.table import Table

def extract_photometry_data(file_path, target_ra, target_dec, tolerance=0.001):
    """从FITS文件中提取所有星的测光数据，并根据目标RA和Dec排除特定目标."""
    hdul = fits.open(file_path)
    data = Table.read(file_path, hdu=1)
    ra = data['ra']  # 假设RA存储在列名为'RA'的列中
    dec = data['dec']  # 假设Dec存储在列名为'Dec'的列中
    instrumental_mag = data['mag']  # 假设测光数据存储在名为'mag'的列中

    star_photometry = {}
    for i in range(len(data)):
        ra_star = ra[i]
        dec_star = dec[i]
        mag_star = instrumental_mag[i]
    
        
        # 组合星的唯一标识，这里使用(RA, Dec)作为唯一标识
        # 为了避免非常接近的星被认为是不同的星，使用四舍五入合并接近的星
        ra_star_rounded = round(ra_star, 3)
        dec_star_rounded = round(dec_star, 3)
        star_id = (ra_star_rounded, dec_star_rounded)
        
        if star_id not in star_photometry:
            star_photometry[star_id] = []
        star_photometry[star_id].append(mag_star)

    return star_photometry

def process_star_files(fits_files, target_ra, target_dec):
    """处理一组FITS文件，提取每颗星的测光数据，排除目标星."""
    total_star_photometry = {}

    for file_path in fits_files:
        star_photometry = extract_photometry_data(file_path, target_ra, target_dec)
        for star_id, mags in star_photometry.items():
            if star_id not in total_star_photometry:
                total_star_photometry[star_id] = []
            total_star_photometry[star_id].extend(mags)

    return total_star_photometry

fits_files = catfnlst_filter2  # 替换为您的FITS文件列表
target_ra = ra_deg  # 目标星的RA
target_dec = dec_deg  # 目标星的Dec

# 处理FITS文件并提取所有星的测光数据，排除目标星
total_star_photometry = process_star_files(fits_files, target_ra, target_dec)

# 找出出现次数最多的前100颗星
star_counts = Counter({star_id: len(mags) for star_id, mags in total_star_photometry.items()})
top_100_stars = star_counts.most_common()[20:50]
next_100_stars = star_counts.most_common()[50:100]

# 计算前100颗星的标准差
star_std_devs = []
for star_id, _ in top_100_stars:
    mags = total_star_photometry[star_id]
    std_dev = np.nanstd(mags)
    star_std_devs.append((star_id, std_dev))

# 按标准差排序，找出标准差最小的前10颗星
star_std_devs.sort(key=lambda x: x[1])
top_10_reference_stars = star_std_devs[:10]

# 输出结果
print("标准差最小的前10颗参考星：")
for star_id, std_dev in top_10_reference_stars:
    ra_star, dec_star = star_id
    print(f"RA={ra_star}, Dec={dec_star} 标准差：{std_dev}")

# 计算后100颗星的标准差
star_std_devs = []
for star_id, _ in next_100_stars:
    mags = total_star_photometry[star_id]
    std_dev = np.nanstd(mags)
    star_std_devs.append((star_id, std_dev))

# 按标准差排序，找出标准差最小的前10颗星
star_std_devs.sort(key=lambda x: x[1])
next_10_reference_stars = star_std_devs[:10]

# 输出结果
print("标准差最小的前10颗检验星：")
for star_id, std_dev in next_10_reference_stars:
    ra_star, dec_star = star_id
    print(f"RA={ra_star}, Dec={dec_star} 标准差：{std_dev}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from collections import Counter

def extract_light_curves(file_path, target_ra, target_dec, reference_stars, tolerance=0.001):
    """从FITS文件中提取测光数据，并使用参考星对目标星进行较差测光."""
    hdul = fits.open(file_path)
    data = Table.read(file_path, hdu=1)

    mjd = hdul[0].header['MJD']
    ra = data['ra']  # RA存储在列名为'Ra'的列中
    dec = data['dec']  # Dec存储在列名为'Dec'的列中
    instrumental_mag = data['mag']  # 测光数据存储在名为'mag'的列中
    mag_error = data['magerr']  # 测光误差存储在名为'magerr'的列中

    # 找到目标星的索引
    target_star_index = np.where((np.abs(ra - target_ra) < tolerance) & (np.abs(dec - target_dec) < tolerance))[0]
    print(target_star_index)
    if len(target_star_index) == 0:
        print(f"未在文件中找到目标星。")
        return None, None, None

    target_star_index = target_star_index[0]

    # 提取目标星的测光和误差
    target_mag = instrumental_mag[target_star_index]
    target_mag_err = mag_error[target_star_index]

    # 对参考星进行较差测光
    reference_mags = []
    reference_mag_errors = []
    matched_ref_stars = set()
    for ref_ra, ref_dec in reference_stars:
        ref_star_indices = np.where((np.abs(ra - ref_ra) < tolerance) & (np.abs(dec - ref_dec) < tolerance))[0]
        for ref_star_index in ref_star_indices:
            if ref_star_index not in matched_ref_stars:
                matched_ref_stars.add(ref_star_index)
                reference_mag = instrumental_mag[ref_star_index]
                reference_mag_err = mag_error[ref_star_index]
                reference_mags.append(reference_mag)
                reference_mag_errors.append(reference_mag_err)
    
    if len(reference_mags) == 0:
        print(f"未在文件中找到任何参考星。")
        return None, None, None

    # 计算相对测光值
    relative_mags = target_mag - np.mean(reference_mags, axis=0)
    relative_mag_err = np.sqrt(target_mag_err**2 + np.mean(reference_mag_errors, axis=0)**2)

    return mjd, relative_mags, relative_mag_err

def extract_ref_light_curves(file_path, ref_ra, ref_dec, check_stars, tolerance=0.001):
    """从FITS文件中提取参考星的测光数据."""
    hdul = fits.open(file_path)
    data = Table.read(file_path, hdu=1)

    mjd = hdul[0].header['MJD']
    ra = data['ra']  # 假设RA存储在列名为'Ra'的列中
    dec = data['dec']  # 假设Dec存储在列名为'Dec'的列中
    instrumental_mag = data['mag']  # 假设测光数据存储在名为'mag'的列中
    mag_error = data['magerr']  # 假设测光误差存储在名为'magerr'的列中

    # 找到参考星的索引
    ref_star_index = np.where((np.abs(ra - ref_ra) < tolerance) & (np.abs(dec - ref_dec) < tolerance))[0]
    if len(ref_star_index) == 0:
        print(f"未在文件中找到参考星。")
        return None, None, None

    ref_star_index = ref_star_index[0]

    # 提取参考星的测光和误差
    reference_mag = instrumental_mag[ref_star_index]
    reference_mag_err = mag_error[ref_star_index]

    # 对检查星进行较差测光
    check_mags = []
    check_mag_errors = []
    matched_check_stars = set()
    for chk_ra, chk_dec in check_stars:
        chk_star_indices = np.where((np.abs(ra - chk_ra) < tolerance) & (np.abs(dec - chk_dec) < tolerance))[0]
        for chk_star_index in chk_star_indices:
            if chk_star_index not in matched_check_stars:
                matched_check_stars.add(chk_star_index)
                check_mag = instrumental_mag[chk_star_index]
                check_mag_err = mag_error[chk_star_index]
                check_mags.append(check_mag)
                check_mag_errors.append(check_mag_err)
    
    if len(check_mags) == 0:
        print(f"未在文件中找到任何检查星。")
        return None, None, None

    # 计算相对测光值
    relative_mags = reference_mag - np.mean(check_mags, axis=0)
    relative_mag_err = np.sqrt(reference_mag_err**2 + np.mean(check_mag_errors, axis=0)**2)

    return mjd, relative_mags, relative_mag_err

def process_fits_files(fits_files, target_ra, target_dec, reference_stars, check_stars):
    """处理一组FITS文件，对每幅图像使用参考星进行较差测光."""
    plt.figure(figsize=(12, 40))

    # 绘制标准星的光变曲线
    colors = plt.cm.viridis(np.linspace(0, 1, len(reference_stars)))  # 使用viridis颜色映射
    for i, (ref_ra, ref_dec) in enumerate(reference_stars):
        color = colors[i]
        offset = i * 0.5  # 增加每个标准星的偏移量，根据索引增加
        stddevs = []
        for file_path in fits_files:
            ref_mjd, ref_relative_mags, ref_relative_mag_err = extract_ref_light_curves(file_path, ref_ra, ref_dec, check_stars)
            if ref_mjd is not None and ref_relative_mags is not None and ref_relative_mag_err is not None:
                plt.errorbar([ref_mjd], [ref_relative_mags + offset], yerr=[ref_relative_mag_err], fmt='o', capsize=3, alpha=0.5, color=color)
                stddevs.append(np.std(ref_relative_mags))

        if stddevs:
            # 在光变曲线左侧标注RA、Dec和标准差
            mean_stddev = np.mean(stddevs)

    # 处理目标星的光变曲线
    target_stddevs = []
    for file_path in fits_files:
        target_mjd, target_relative_mags, target_relative_mag_err = extract_light_curves(file_path, target_ra, target_dec, reference_stars)
        if target_mjd is not None and target_relative_mags is not None and target_relative_mag_err is not None:
            plt.errorbar([target_mjd], [target_relative_mags - 0.5], yerr=[target_relative_mag_err], fmt='o', capsize=3, color='red')
            target_stddevs.append(np.std(target_relative_mags))

    # 在目标星光变曲线左侧标注RA、Dec和标准差
    if target_stddevs:
        mean_target_stddev = np.mean(target_stddevs)

    plt.xlabel('MJD')
    plt.ylabel('$m_{relative}$')
    plt.title(f'Light curves of target star and reference stars')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.savefig('light_curves_2.png')
    plt.show()

fits_files = catfnlst_filter2  # 替换为您的FITS文件列表
target_ra = ra_deg  # 目标星的RA
target_dec = dec_deg  # 目标星的Dec

# 找出出现次数最多的前10颗星作为参考星
reference_stars = list(set([star_id for star_id, _ in top_10_reference_stars]))
check_stars = list(set([star_id2 for star_id2, _ in next_10_reference_stars]))

# 处理FITS文件并进行较差测光
process_fits_files(fits_files, target_ra, target_dec, reference_stars, check_stars)